In [25]:
import os

In [26]:
%pwd

'c:\\project\\chicken-disease-classification'

In [31]:
os.chdir('C:\project\chicken-disease-classification')

In [32]:
%pwd

'C:\\project\\chicken-disease-classification'

In [33]:
from dataclasses import dataclass
from  pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url:str
    local_data_file: Path
    unzip_dir: Path

In [34]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [35]:
from cnnClassifier import logger
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        
        
        self.config=read_yaml(CONFIG_FILE_PATH)
        logger.info('config file loaded')
        self.params=read_yaml(params_filepath)
        logger.info('param file loaded')
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [36]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [46]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file 
            )
            logger.info(f'{filename} download with following info \n{headers}')
        else:
            logger.info((f'file already exists of size {os.path.getsize(Path(self.config.local_data_file))}'))

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    
    

In [47]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-29 11:36:35,805: INFO: common: yaml file: config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-05-29 11:36:35,806: INFO: 1975627020: config file loaded]
[2024-05-29 11:36:35,808: INFO: common: yaml file: params.yaml loaded successfully]
{'key': 'val', 'another_key': 'another_val'}
[2024-05-29 11:36:35,808: INFO: 1975627020: param file loaded]
[2024-05-29 11:36:35,808: INFO: common: created directory at: artifacts]
[2024-05-29 11:36:35,808: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-29 11:36:35,816: INFO: 1436007680: file already exists of size 11616915]
